Project code



In [53]:
# Scrape data for historical funding figures

import requests
from bs4 import BeautifulSoup

# Get the webpage
url = 'https://www.uksport.gov.uk/our-work/investing-in-sport/historical-funding-figures'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Find the heading before the table
heading = soup.find('h3', string="Summer Olympic Sports")

# Get the next table after the heading
table = heading.find_next('table') 

# Extract and save the data

rows = table.find_all('tr')

data = []
for row in rows:
    cells = row.find_all(['th', 'td'])
    print([cell.get_text(strip=True) for cell in cells])


['Sport', 'London', 'Rio', 'Tokyo**', 'Paris*']
['Archery', '£3,479,900', '£2,473,437', '£1,122,879', '£2,632,674']
['Artistic Swimming', '£2,686,800', '£1,011,290', '£251,000', '£467,250']
['Athletics', '£20,613,100', '£21,820,808', '£22,416,808', '£22,175,520']
['Badminton', '£5,979,860', '£4,922,229', '£889,793', '£3,322,558']
['Baseball', '-', '-', '£30,500', '£176,250']
['Basketball', '£7,127,800', '£1,927,061', '£330,500', '£1,350,000']
['Boxing', '£7,551,900', '£11,151,737', '£11,606,057', '£11,597,507']
['Breaking', '-', '-', '-', '£135,000']
['Canoeing', '£12,981,800', '£16,293,651', '£15,765,693', '£13,081,836']
['Cycling', '£20,613,900', '£23,966,916', '£24,559,306', '£29,172,590']
['Diving', '£5,190,700', '£6,133,760', '£7,172,659', '£8,860,542']
['Equestrian', '£10,580,200', '£14,397,300', '£11,951,195', '£11,763,376']
['Fencing', '£2,132,346', '£3,593,344', '£341,631', '£1,542,015']
['Gymnastics', '£8,734,100', '£11,791,228', '£13,285,417', '£13,180,313']
['Handball', '£2